In [1]:
import os.path
import zipfile
fname = 'mixesdb_all_sets_featurized.csv'
if not os.path.isfile(fname):
    zip_ref = zipfile.ZipFile(fname+".zip", 'r')
    zip_ref.extractall('.')
    zip_ref.close()
with open(fname) as f:
    print(f.read().split('\n',2)[:2])


[',artist,bool_S_Acid,bool_S_Deep,bool_S_Detroit,bool_S_Disco,bool_S_Drum_&_Bass,bool_S_Dub,bool_S_Electro,bool_S_Garage,bool_S_Hard,bool_S_Hip_Hop,bool_S_House,bool_S_IDM,bool_S_Industrial,bool_S_Leftfield,bool_S_Minimal,bool_S_Old_School,bool_S_Progressive,bool_S_Psytrance,bool_S_Tech_House,bool_S_Techno,bool_S_Trance,label,set_duration,set_genres,set_metadata,set_title,set_year,track,track_order,track_timing,bool_S_complete', '0,Moonbeam,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,,1:00:00,"[\'House\', \'Hard Trance\']","[\'2011\', \'ASYS\', ""Filter\'d"", \'Hard Trance\', \'House\', \'Tracklist: complete\']",2011-12-30 - ASYS - Filter\'d 71 (Best Of 2011),2011,Hate Is The Killer (Arty Remix),0,,True']


In [2]:
boooools = "bool_S_Acid,bool_S_Deep,bool_S_Detroit,bool_S_Disco,bool_S_Drum_&_Bass,bool_S_Dub,bool_S_Electro,bool_S_Garage,bool_S_Hard,bool_S_Hip_Hop,bool_S_House,bool_S_IDM,bool_S_Industrial,bool_S_Leftfield,bool_S_Minimal,bool_S_Old_School,bool_S_Progressive,bool_S_Psytrance,bool_S_Tech_House,bool_S_Techno,bool_S_Trance"\
                .split(',')

feature_amount = len(boooools)
print(feature_amount)


21


In [3]:
import pandas
tracklist_df = pandas.read_csv(fname)

In [4]:
def get_tracklists(df):
    tracklist = []
    album = ""
    for i, track in df.iterrows():
        if album == "":
            album = track['set_title']
        if track['set_title'] != album:
            yield album, tracklist
            album = track['set_title']
            tracklist = []
        tracklist += [track]
    yield album, tracklist


def to_vector(track):
    return [int(bool(track[b])) for b in boooools]

def to_matrix(tracklist):
    return [to_vector(track) for track in tracklist]

def seq_matrices(df):
    for album, tracklist in get_tracklists(df):
        yield to_matrix(tracklist)

In [5]:
def group_matrices_by_length(df):
    groups = {}
    for m in seq_matrices(df):
        if len(m) in groups:
            groups[len(m)] += [m]
        else:
            groups[len(m)] = [m]
    return groups

In [6]:
groups = group_matrices_by_length(tracklist_df)

In [7]:

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

model = Sequential()
# batch, time_steps, features
# input_shape = (time_steps, features)
# time_steps = variable = -1 or None
model.add(LSTM(200, input_shape=(None, feature_amount), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(200, return_sequences=True))
model.add(Dense(feature_amount))
print(model.summary())

model.compile(optimizer="adam", loss="MSE")


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 200)         177600    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 200)         320800    
_________________________________________________________________
dense_1 (Dense)              (None, None, 21)          4221      
Total params: 502,621
Trainable params: 502,621
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
import numpy as np

print(len(groups.items()))
print([(l,np.shape(np.array(m))) for l, m in list(groups.items())[:10]])

155
[(13, (1907, 13, 21)), (5, (407, 5, 21)), (19, (1042, 19, 21)), (32, (347, 32, 21)), (24, (812, 24, 21)), (27, (612, 27, 21)), (28, (520, 28, 21)), (11, (1918, 11, 21)), (10, (1657, 10, 21)), (16, (1304, 16, 21))]


In [11]:
# train RNN
def train(model,epochs,verbose):
    for epoch in range(epochs):
        print("epoch", epoch)
        for l, matrices in groups.items():
            print("tracklists of length", l, "amount", len(matrices))
            if len(matrices) < 1 or l <= 1: continue
            batch = np.array(matrices)
            x = batch[:,:-1]
            y = batch[:,1:]
            model.fit(x,y,epochs = 1, verbose = verbose)

In [13]:
"""train"""
train(model,10,0)

epoch 0
tracklists of length 13 amount 1907
tracklists of length 5 amount 407
tracklists of length 19 amount 1042
tracklists of length 32 amount 347
tracklists of length 24 amount 812
tracklists of length 27 amount 612
tracklists of length 28 amount 520
tracklists of length 11 amount 1918
tracklists of length 10 amount 1657
tracklists of length 16 amount 1304
tracklists of length 17 amount 1146
tracklists of length 65 amount 32
tracklists of length 23 amount 905
tracklists of length 33 amount 252
tracklists of length 20 amount 1185
tracklists of length 14 amount 1673
tracklists of length 29 amount 476
tracklists of length 36 amount 228
tracklists of length 18 amount 998
tracklists of length 22 amount 954
tracklists of length 12 amount 2083
tracklists of length 26 amount 706
tracklists of length 15 amount 1604
tracklists of length 31 amount 358
tracklists of length 21 amount 931
tracklists of length 48 amount 65
tracklists of length 43 amount 120
tracklists of length 25 amount 749
track

tracklists of length 79 amount 17
tracklists of length 92 amount 3
tracklists of length 60 amount 32
tracklists of length 116 amount 3
tracklists of length 102 amount 6
tracklists of length 113 amount 2
tracklists of length 93 amount 8
tracklists of length 118 amount 2
tracklists of length 177 amount 1
tracklists of length 78 amount 15
tracklists of length 94 amount 6
tracklists of length 85 amount 8
tracklists of length 84 amount 10
tracklists of length 77 amount 17
tracklists of length 103 amount 6
tracklists of length 111 amount 5
tracklists of length 87 amount 9
tracklists of length 95 amount 4
tracklists of length 176 amount 1
tracklists of length 156 amount 2
tracklists of length 211 amount 1
tracklists of length 83 amount 9
tracklists of length 104 amount 4
tracklists of length 75 amount 10
tracklists of length 88 amount 11
tracklists of length 98 amount 7
tracklists of length 101 amount 4
tracklists of length 96 amount 5
tracklists of length 81 amount 7
tracklists of length 105

tracklists of length 14 amount 1673
tracklists of length 29 amount 476
tracklists of length 36 amount 228
tracklists of length 18 amount 998
tracklists of length 22 amount 954
tracklists of length 12 amount 2083
tracklists of length 26 amount 706
tracklists of length 15 amount 1604
tracklists of length 31 amount 358
tracklists of length 21 amount 931
tracklists of length 48 amount 65
tracklists of length 43 amount 120
tracklists of length 25 amount 749
tracklists of length 45 amount 109
tracklists of length 69 amount 19
tracklists of length 9 amount 827
tracklists of length 72 amount 17
tracklists of length 35 amount 237
tracklists of length 66 amount 30
tracklists of length 30 amount 457
tracklists of length 46 amount 91
tracklists of length 37 amount 215
tracklists of length 40 amount 164
tracklists of length 106 amount 6
tracklists of length 39 amount 176
tracklists of length 61 amount 31
tracklists of length 7 amount 532
tracklists of length 4 amount 300
tracklists of length 67 amo

tracklists of length 87 amount 9
tracklists of length 95 amount 4
tracklists of length 176 amount 1
tracklists of length 156 amount 2
tracklists of length 211 amount 1
tracklists of length 83 amount 9
tracklists of length 104 amount 4
tracklists of length 75 amount 10
tracklists of length 88 amount 11
tracklists of length 98 amount 7
tracklists of length 101 amount 4
tracklists of length 96 amount 5
tracklists of length 81 amount 7
tracklists of length 105 amount 5
tracklists of length 131 amount 2
tracklists of length 91 amount 4
tracklists of length 228 amount 1
tracklists of length 2 amount 248
tracklists of length 132 amount 3
tracklists of length 121 amount 4
tracklists of length 1 amount 58
tracklists of length 117 amount 1
tracklists of length 119 amount 2
tracklists of length 184 amount 1
tracklists of length 128 amount 1
tracklists of length 141 amount 1
tracklists of length 108 amount 5
tracklists of length 127 amount 2
tracklists of length 125 amount 3
tracklists of length 1

tracklists of length 72 amount 17
tracklists of length 35 amount 237
tracklists of length 66 amount 30
tracklists of length 30 amount 457
tracklists of length 46 amount 91
tracklists of length 37 amount 215
tracklists of length 40 amount 164
tracklists of length 106 amount 6
tracklists of length 39 amount 176
tracklists of length 61 amount 31
tracklists of length 7 amount 532
tracklists of length 4 amount 300
tracklists of length 67 amount 26
tracklists of length 44 amount 121
tracklists of length 53 amount 72
tracklists of length 34 amount 260
tracklists of length 63 amount 27
tracklists of length 3 amount 404
tracklists of length 38 amount 191
tracklists of length 42 amount 143
tracklists of length 41 amount 161
tracklists of length 47 amount 92
tracklists of length 68 amount 23
tracklists of length 62 amount 17
tracklists of length 58 amount 30
tracklists of length 57 amount 36
tracklists of length 6 amount 388
tracklists of length 8 amount 556
tracklists of length 59 amount 31
trac

tracklists of length 91 amount 4
tracklists of length 228 amount 1
tracklists of length 2 amount 248
tracklists of length 132 amount 3
tracklists of length 121 amount 4
tracklists of length 1 amount 58
tracklists of length 117 amount 1
tracklists of length 119 amount 2
tracklists of length 184 amount 1
tracklists of length 128 amount 1
tracklists of length 141 amount 1
tracklists of length 108 amount 5
tracklists of length 127 amount 2
tracklists of length 125 amount 3
tracklists of length 134 amount 1
tracklists of length 114 amount 2
tracklists of length 186 amount 1
tracklists of length 139 amount 2
tracklists of length 130 amount 3
tracklists of length 167 amount 1
tracklists of length 219 amount 1
tracklists of length 129 amount 1
tracklists of length 89 amount 3
tracklists of length 244 amount 1
tracklists of length 303 amount 2
tracklists of length 122 amount 2
tracklists of length 238 amount 1
tracklists of length 213 amount 1
tracklists of length 90 amount 3
tracklists of leng

tracklists of length 53 amount 72
tracklists of length 34 amount 260
tracklists of length 63 amount 27
tracklists of length 3 amount 404
tracklists of length 38 amount 191
tracklists of length 42 amount 143
tracklists of length 41 amount 161
tracklists of length 47 amount 92
tracklists of length 68 amount 23
tracklists of length 62 amount 17
tracklists of length 58 amount 30
tracklists of length 57 amount 36
tracklists of length 6 amount 388
tracklists of length 8 amount 556
tracklists of length 59 amount 31
tracklists of length 56 amount 38
tracklists of length 99 amount 6
tracklists of length 74 amount 12
tracklists of length 100 amount 7
tracklists of length 55 amount 40
tracklists of length 50 amount 69
tracklists of length 71 amount 26
tracklists of length 80 amount 19
tracklists of length 109 amount 1
tracklists of length 73 amount 12
tracklists of length 51 amount 67
tracklists of length 54 amount 39
tracklists of length 107 amount 6
tracklists of length 52 amount 59
tracklists 

In [14]:
%%time
"""test"""
train(model,1,1)

epoch 0
tracklists of length 13 amount 1907
Epoch 1/1
1907/1907 [==============================] - 4s - loss: 1.6266e-04     
tracklists of length 5 amount 407
Epoch 1/1
407/407 [==============================] - 0s - loss: 1.1675e-04     
tracklists of length 19 amount 1042
Epoch 1/1
1042/1042 [==============================] - 3s - loss: 3.5469e-05     
tracklists of length 32 amount 347
Epoch 1/1
347/347 [==============================] - 1s - loss: 2.9893e-05     
tracklists of length 24 amount 812
Epoch 1/1
812/812 [==============================] - 3s - loss: 2.5035e-05     
tracklists of length 27 amount 612
Epoch 1/1
612/612 [==============================] - 2s - loss: 2.3735e-05     
tracklists of length 28 amount 520
Epoch 1/1
520/520 [==============================] - 2s - loss: 2.2769e-05     
tracklists of length 11 amount 1918
Epoch 1/1
1918/1918 [==============================] - 3s - loss: 3.3281e-05     
tracklists of length 10 amount 1657
Epoch 1/1
1657/1657 [=======

6/6 [==============================] - 0s - loss: 2.9182e-05
tracklists of length 52 amount 59
Epoch 1/1
59/59 [==============================] - 0s - loss: 2.4043e-05     
tracklists of length 82 amount 14
Epoch 1/1
14/14 [==============================] - 0s - loss: 1.6659e-05
tracklists of length 70 amount 15
Epoch 1/1
15/15 [==============================] - 0s - loss: 2.7177e-05
tracklists of length 49 amount 64
Epoch 1/1
64/64 [==============================] - 0s - loss: 2.4319e-05     
tracklists of length 64 amount 30
Epoch 1/1
30/30 [==============================] - 0s - loss: 2.8382e-05
tracklists of length 126 amount 3
Epoch 1/1
3/3 [==============================] - 0s - loss: 4.7564e-06
tracklists of length 76 amount 12
Epoch 1/1
12/12 [==============================] - 0s - loss: 1.0179e-05
tracklists of length 183 amount 1
Epoch 1/1
1/1 [==============================] - 0s - loss: 3.0340e-06
tracklists of length 115 amount 2
Epoch 1/1
2/2 [============================

1/1 [==============================] - 0s - loss: 3.1223e-04
tracklists of length 161 amount 1
Epoch 1/1
1/1 [==============================] - 0s - loss: 1.5102e-04
tracklists of length 478 amount 1
Epoch 1/1
1/1 [==============================] - 0s - loss: 7.0351e-04
tracklists of length 165 amount 1
Epoch 1/1
1/1 [==============================] - 0s - loss: 4.3418e-04
tracklists of length 136 amount 1
Epoch 1/1
1/1 [==============================] - 0s - loss: 0.0013
CPU times: user 6min 10s, sys: 1min 8s, total: 7min 19s
Wall time: 2min 2s


In [18]:
model.save_weights("mixesdb_rnn.h5")

In [ ]:
model.load_weights("mixesdb_rnn.h5")

In [16]:

# generate 1 list

import random

def random_tracks(df, amount=1):
    rs = [random.randint(0,df.shape[0]) for _ in range(amount)]
    return [df.iloc[r] for r in rs]

def metric(vector1, vector2):
    v1, v2 = np.array(vector1), np.array(vector2)
    d = (v1 - v2)
    return np.sum(d * d)

def nearest_neighbor(vec, tracks):
    return min(tracks, key=lambda t: metric(vec, to_vector(t)))

def generate_next(tracks, df, model, params):
    predicted = model.predict(np.array([to_matrix(tracks)]))[0][-1]
    neighbor = nearest_neighbor(predicted, [track for i, track in df.iterrows()])
    return neighbor

def generate_next_v2(tracks, df, model, params):
    predicted = model.predict(np.array([to_matrix(tracks)]))[0][-1]
    neighbor = nearest_neighbor(predicted, random_tracks(df,params['tournament_size']))
    return neighbor



def generate_list(df, model, params):
    tracks = random_tracks(df, 1)
    yield tracks[-1]
    for i in range(1, params['amount']):
        tracks += [generate_next_v2(tracks, df, model, params)]
        yield tracks[-1]


In [17]:
params = {'amount':15, 'tournament_size': 1000}

f = open('mixesdb_gen.txt','a+')

for i in range(5):
    gen = generate_list(tracklist_df,model, params)
    for track in gen:
        print(track['artist'],'-',track['track'])
        f.write(track['artist'] + '-' + track['track'] + "\n")
    print("--------------")
    f.write("--------------")

f.close()

File:1994-07-07 - Stretch & Bobbito
Wu-Tang Clan - C.R.E.A.M
Snow - Informer
? - ?
Merlin - You Blow My Mind
Kontra K - Soldaten 2.0
Common - It's Your World Pt. 2
Ape Drums Feat. Andre 'Suku' Gray - Deva
Bloods And Crips - Piru Love
Gang Starr - The ? Remains
Outkast - 13th Floor Growing Old
Morpheground - Palm Shade Tree
Lethal Bizzle - Fester Skank
Ashanti - Movies (Murlo Refix)
Stylo G - Yuh Zimmie
--------------
Harvey Sutherland - Speaking Your Language
Detroit Swindle - Unfinished Business (CRUE Remix)
95 North - Forever Underground (The Bass Dub)
M A N I K Feat. Cari Golden - Recourse
Jimmy Edgar - Strike
Family Of Few - Intervoles
? - ?
The Carter Bros - Ritual Business
Leman & Dieckmann - Reset
Mashti & Jean Von Baden - Man Of Trouble (Djuma Soundsystem Remix)
Trueby Trio - New Music
Axel Boman - Nattsudd
Mark Fanciulli - Impact
Joeski - Earth Cry
? - ?
--------------
Few Nolder - Twin
David Mayer - Celsius
UCC (Urban Cookie Collective) - Rest Of My Love (Dodo Bassburger Dub)